<a href="https://colab.research.google.com/github/vggls/msc_thesis_medical_xai/blob/main/experiments/ham10k_resnet50/MaxSensitivity_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Imports**

In [ ]:
pip install grad-cam

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for grad-cam: filename=grad_cam-1.4.6-py3-none-any.whl size=38261 sha256=cc1ca4a6fcb5cb26b6648aa3ab8a127196065720ed85b02cabc7d8f60563a132
  Stored in directory: /root/.cache/pip/wheels/53/02/43/1f75726b5c28459596067ad91e36951463c01273eef661f09f
Successfully built grad-cam


In [ ]:
import shutil
import pickle

import torch
from torchvision  import transforms

from pytorch_grad_cam import GradCAM, HiResCAM

In [ ]:
# custom written code
from ham10k import map_images_to_paths, create_datasets, summaries
from max_sensitivity import sample_eps_Inf, get_explanation, get_exp_sens, MaxSensitivity_Dataset, plot_scores_frequency, scores_boxplot

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **2. Download from Kaggle**


In [ ]:
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download kmader/skin-cancer-mnist-ham10000

100% 5.19G/5.20G [00:33<00:00, 181MB/s]
100% 5.20G/5.20G [00:33<00:00, 165MB/s]


In [ ]:
! unzip skin-cancer-mnist-ham10000.zip

Η έξοδος ροής περικόπηκε στις τελευταίες 5000 γραμμές.
  inflating: ham10000_images_part_2/ISIC_0029325.jpg  
  inflating: ham10000_images_part_2/ISIC_0029326.jpg  
  inflating: ham10000_images_part_2/ISIC_0029327.jpg  
  inflating: ham10000_images_part_2/ISIC_0029328.jpg  
  inflating: ham10000_images_part_2/ISIC_0029329.jpg  
  inflating: ham10000_images_part_2/ISIC_0029330.jpg  
  inflating: ham10000_images_part_2/ISIC_0029331.jpg  
  inflating: ham10000_images_part_2/ISIC_0029332.jpg  
  inflating: ham10000_images_part_2/ISIC_0029333.jpg  
  inflating: ham10000_images_part_2/ISIC_0029334.jpg  
  inflating: ham10000_images_part_2/ISIC_0029335.jpg  
  inflating: ham10000_images_part_2/ISIC_0029336.jpg  
  inflating: ham10000_images_part_2/ISIC_0029337.jpg  
  inflating: ham10000_images_part_2/ISIC_0029338.jpg  
  inflating: ham10000_images_part_2/ISIC_0029339.jpg  
  inflating: ham10000_images_part_2/ISIC_0029340.jpg  
  inflating: ham10000_images_part_2/ISIC_0029341.jpg  
  inflatin

# **3. Test dataset**

In [ ]:
#shutil.rmtree('./test_dataset')

In [ ]:
# to be applied to training data
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),          #recommended size
    transforms.RandomHorizontalFlip(),      #data augmentation
    transforms.RandomVerticalFlip(),        #data augmentation
    transforms.RandomRotation(degrees=20),  #data augmentation
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor([0.5, 0.5, 0.5]), torch.Tensor([0.5, 0.5, 0.5]))
])
# to be applied to validation and test data
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor([0.5, 0.5, 0.5]), torch.Tensor([0.5, 0.5, 0.5]))
])

In [ ]:
dataframe = map_images_to_paths()

In [ ]:
train_dataset, validation_dataset, test_dataset= create_datasets(dataframe = dataframe,
                                                                 train_transforms = train_transforms,
                                                                 test_transforms = test_transforms)

No lesion overlap among training, validation and test datasets!
All images succesfully distributed in the training, validation and test datasets!


In [ ]:
# do not need them in this notebook
shutil.rmtree('./training_dataset')
shutil.rmtree('./validation_dataset')

# **4. Load model**

In [ ]:
model_path = './drive/MyDrive/Colab_Notebooks/dataset_models/HAM10000/Models/'
resnet50 = torch.load(model_path + 'ham10k_resnet50.pt')

In [ ]:
resnet50 = resnet50.cuda()
resnet50 = resnet50.eval()

# **5. Experiment function**

In [ ]:
def run_experiment(radius, labels, samples_per_img, xai):

    assert xai in ['gradcam', 'hirescam']

    if xai == 'gradcam':
      cam_instance = GradCAM(model=resnet50, target_layers=[resnet50.layer4[2].conv3], use_cuda=True)
    elif xai == 'hirescam':
      cam_instance = HiResCAM(model=resnet50, target_layers=[resnet50.layer4[2].conv3], use_cuda=True)

    print('radius:{} - labels:{}'.format(radius, labels))

    dataset = [(img, label) for (img, label) in test_dataset if (label in labels)]
    _, data_scores = MaxSensitivity_Dataset(dataset, resnet50, cam_instance, radius, samples_per_img)

    drive_path = './drive/MyDrive/Colab_Notebooks/dataset_models/HAM10000/Max_Sensitivity/exp_ResNet50/'

    if xai == 'gradcam':
      with open(drive_path + 'maxsens_resnet50_gradcam_radius_{}_labels_{}.pickle'.format(radius, labels), 'wb') as f:
        pickle.dump(data_scores, f)
    elif xai == 'hirescam':
      with open(drive_path + 'maxsens_resnet50_hirescam_radius_{}_labels_{}.pickle'.format(radius, labels), 'wb') as f:
        pickle.dump(data_scores, f)

    return data_scores

# **6. GradCAM**

## **radius 0.05**

In [ ]:
data_scores = run_experiment(radius=0.05, labels=[0,1,2,3,4,5,6], samples_per_img=20, xai='gradcam')

radius:0.05 - labels:[0, 1, 2, 3, 4, 5, 6]
Total time: 492.37465238571167 secs
Correctly predicted images: 1405/1992
Avg secs per image:  0.35


In [ ]:
sum(data_scores)

57126.119441986084

## **radius 0.1**

In [ ]:
data_scores = run_experiment(radius=0.1, labels=[0,1,2,3,4,5,6], samples_per_img=20, xai='gradcam')

radius:0.1 - labels:[0, 1, 2, 3, 4, 5, 6]
Total time: 458.54472064971924 secs
Correctly predicted images: 1405/1992
Avg secs per image:  0.33


In [ ]:
sum(data_scores)

75947.1097984314

## **radius 0.2**

In [ ]:
data_scores = run_experiment(radius=0.2, labels=[0,1,2,3,4,5,6], samples_per_img=30, xai='gradcam')

radius:0.2 - labels:[0, 1, 2, 3, 4, 5, 6]
Total time: 722.0203359127045 secs
Correctly predicted images: 1405/1992
Avg secs per image:  0.51


In [ ]:
sum(data_scores)

96303.12244415283

## **radius 0.3**

In [ ]:
data_scores = run_experiment(radius=0.3, labels=[0,1,2,3,4,5,6], samples_per_img=30, xai='gradcam')

radius:0.3 - labels:[0, 1, 2, 3, 4, 5, 6]
Total time: 673.3842821121216 secs
Correctly predicted images: 1405/1992
Avg secs per image:  0.48


In [ ]:
sum(data_scores)

103252.98023986816

## **radius 0.4**

In [ ]:
data_scores = run_experiment(radius=0.4, labels=[0,1,2,3,4,5,6], samples_per_img=40, xai='gradcam')

radius:0.4 - labels:[0, 1, 2, 3, 4, 5, 6]
Total time: 943.73845911026 secs
Correctly predicted images: 1405/1992
Avg secs per image:  0.67


In [ ]:
sum(data_scores)

108743.63329315186

## **radius 0.5**

In [ ]:
data_scores = run_experiment(radius=0.5, labels=[0,1,2,3,4,5,6], samples_per_img=40, xai='gradcam')

radius:0.5 - labels:[0, 1, 2, 3, 4, 5, 6]
Total time: 954.8949048519135 secs
Correctly predicted images: 1405/1992
Avg secs per image:  0.68


In [ ]:
sum(data_scores)

110897.94624900818

# **7. HiResCAM**

## **radius 0.05**

In [ ]:
data_scores = run_experiment(radius=0.05, labels=[0,1,2,3,4,5,6], samples_per_img=20, xai='hirescam')

radius:0.05 - labels:[0, 1, 2, 3, 4, 5, 6]
Total time: 481.7357647418976 secs
Correctly predicted images: 1405/1992
Avg secs per image:  0.34


In [ ]:
sum(data_scores)

36177.390320301056

## **radius 0.1**

In [ ]:
data_scores = run_experiment(radius=0.1, labels=[0,1,2,3,4,5,6], samples_per_img=20, xai='hirescam')

radius:0.1 - labels:[0, 1, 2, 3, 4, 5, 6]
Total time: 457.3932251930237 secs
Correctly predicted images: 1405/1992
Avg secs per image:  0.33


In [ ]:
sum(data_scores)

48525.16718959808

## **radius 0.2**

In [ ]:
data_scores = run_experiment(radius=0.2, labels=[0,1,2,3,4,5,6], samples_per_img=30, xai='hirescam')

radius:0.2 - labels:[0, 1, 2, 3, 4, 5, 6]
Total time: 711.1903655529022 secs
Correctly predicted images: 1405/1992
Avg secs per image:  0.51


In [ ]:
sum(data_scores)

61690.83666610718

## **radius 0.3**

In [ ]:
data_scores = run_experiment(radius=0.3, labels=[0,1,2,3,4,5,6], samples_per_img=30, xai='hirescam')

radius:0.3 - labels:[0, 1, 2, 3, 4, 5, 6]
Total time: 696.3609073162079 secs
Correctly predicted images: 1405/1992
Avg secs per image:  0.5


In [ ]:
sum(data_scores)

67099.48822593689

## **radius 0.4**

In [ ]:
data_scores = run_experiment(radius=0.4, labels=[0,1,2,3,4,5,6], samples_per_img=40, xai='hirescam')

radius:0.4 - labels:[0, 1, 2, 3, 4, 5, 6]
Total time: 944.1389954090118 secs
Correctly predicted images: 1405/1992
Avg secs per image:  0.67


In [ ]:
sum(data_scores)

72461.25864219666

## **radius 0.5**

In [ ]:
data_scores = run_experiment(radius=0.5, labels=[0,1,2,3,4,5,6], samples_per_img=40, xai='hirescam')

radius:0.5 - labels:[0, 1, 2, 3, 4, 5, 6]


In [ ]:
sum(data_scores)

76915.53720283508